# Data Cleaning

In this notebook I will:

- Remove duplicate results.
- Fill missing values that can be filled.
- Drop missing values that can't be filled.
- Remove non-informative columns.

In [132]:
import numpy as np
import pandas as pd

from pathlib import Path

In [133]:
DATA_PATH = Path('../data/raw/')
titanic_train = pd.read_csv(DATA_PATH/'train.csv')
titanic_test = pd.read_csv(DATA_PATH/'test.csv')

## Duplicates

In [134]:
print(f'Training set duplicates: {titanic_train.duplicated().sum()}')
print(f'Test set duplicates: {titanic_test.duplicated().sum()}')

Training set duplicates: 0
Test set duplicates: 0


There are no duplicates in our data so no work to be done here.

## Fill Missing Values

### Embarked

We have missing embarked values in our training set. The most sensible option, as there is fairly few missing values, is to fill the missing values with the most common response for embarked. This is Southampton (S).

In [135]:
titanic_train['Embarked'] = titanic_train['Embarked'].fillna('S')

### Fare

We have one missing value in our test data for Fare. One sensible solution would be to fill this missing value based on the median fare for passengers in the same class. I am using the median here as the data contains some outliers.

In [136]:
# Calculate the median fare for each Pclass
fare_medians = titanic_train.groupby('Pclass')['Fare'].median()
# Replace missing values with median for passengers Pclass
titanic_test['Fare'] = titanic_test.apply(
    lambda x: np.nan_to_num(x['Fare'],nan=fare_medians[x['Pclass']]),
    axis=1,
)

### Cabin

The cabin seems to be a column that we could help to engineer a position on the boat for some passengers so we don't want to get rid of it. I will fill any missing values with a placeholder that can be used to engineer a feature from later. 

In [137]:
print(f"Training data missing cabins ratio: {titanic_train['Cabin'].isnull().sum()/len(titanic_train)}")
print(f"Test data missing cabins: {titanic_test['Cabin'].isnull().sum()/len(titanic_test)}")

Training data missing cabins ratio: 0.7710437710437711
Test data missing cabins: 0.7822966507177034


In [138]:
titanic_train['Cabin'] = titanic_train['Cabin'].fillna('N/A')
titanic_test['Cabin'] = titanic_test['Cabin'].fillna('N/A')

### Age

Age could be an important predictor for survival so we will need to decide how best to fill any missing values. We have a few options:
- Fill using mean/median.
- Use another column with high correlation to fill missing values.
- Use a machine learning algorithm to predict age based on other columns.

We have already computed the mean/median above so I'll have a look at correlations between other columns and Age now.

In [139]:
print(f"Training data missing age values: {titanic_train['Age'].isnull().sum()}")
print(f"Test data missing age values: {titanic_test['Age'].isnull().sum()}")

Training data missing age values: 177
Test data missing age values: 86


In [140]:
titanic_train.corr()['Age'].sort_values(ascending=False)[1:]

Fare           0.096067
PassengerId    0.036847
Survived      -0.077221
Parch         -0.189119
SibSp         -0.308247
Pclass        -0.369226
Name: Age, dtype: float64

There are no columns with great correlation to age but we do learn some interesting facts:

- There is a negative correlation between Pclass and Age. This would indicate that lower classes had a higher number of older passengers.
- There is a negative correlation between Age and SibSp. This would make sense as SibSp was a count of number of siblings or spouses on the ship. As you can only have one spouse any number higher than one would indicate a passenger travelling with a sibling. It would make sense for it to be more likely to be a younger passenger (child) if the passenger was travelling with siblings.
- There was a negative correlation between Age and Parch. This also makes sense as if a child was travelling with parents, Parch would be higher.

From this information, I think the best way to fill in the missing data would be by using a machine learning algorithm.

In [141]:
def get_age_predictors(data_frame):
    """
    Returns a data frame with only the required columns needed for the age predicting model.
    """
    needed_columns = ['Fare', 'Parch', 'SibSp', 'Pclass', 'Sex', 'Age']
    age_predictors_df = pd.get_dummies(data_frame[needed_columns], drop_first=True)
    
    return age_predictors_df

In [142]:
# Create our training data for Age
age_predictors_df = get_age_predictors(titanic_train).dropna()
age_predictors, age_target = age_predictors_df.drop('Age', axis=1), age_predictors_df['Age']
age_predictors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 5 columns):
Fare        714 non-null float64
Parch       714 non-null int64
SibSp       714 non-null int64
Pclass      714 non-null int64
Sex_male    714 non-null uint8
dtypes: float64(1), int64(3), uint8(1)
memory usage: 28.6 KB


The problem we face now is which machine learning model do we use to predict the age. We only have a data set which has 714 entries, which limits our options. We also don't want a too complex model that takes a long time to train. I will try the lasso and elastic net models with cross validation to model the age. I'm using these methods as they produce a model with less variance.

In [143]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(cv=5, random_state=0).fit(age_predictors, age_target)
print(lasso.score(age_predictors, age_target))
print(lasso.coef_)

0.2365634914181899
[-0.01730285 -0.74636471 -3.75835274 -6.85956611  2.91621768]


In [144]:
from sklearn.linear_model import ElasticNetCV

enet = ElasticNetCV(cv=10, random_state=0).fit(age_predictors, age_target)
print(enet.score(age_predictors, age_target))
print(enet.coef_)

0.2325918121972903
[-0.010343   -0.96934404 -3.53365767 -5.92987943  2.08938831]


The lasso model seems to give a slightly better R<sup>2</sup> score and so I will use this to predict the missing age values.

In [145]:
def predict_missing_age(dataframe, model):
    age_predictors_df = get_age_predictors(dataframe)
    predictors = ['Fare','Parch','SibSp','Pclass','Sex_male']
    dataframe['Age'] = age_predictors_df.apply(
        lambda x: np.nan_to_num(x['Age'], nan=max(model.predict(x[predictors].to_numpy().reshape(1, -1))[0],0)),
        axis=1,
    )
    return dataframe

In [146]:
titanic_train = predict_missing_age(titanic_train, lasso)
titanic_test = predict_missing_age(titanic_test, lasso)

I used the lasso model to predict the missing ages. If the model returned a negative value I set this to zero. I think this is a much better way of predicting the age of each passenger than using the mean or median as it uses more of the information available to me.

Let's check we have filled any missing values.

In [147]:
print(f'Training data missing values: {titanic_train.isnull().sum().sum()}')
print(f'Test data missing values: {titanic_test.isnull().sum().sum()}')

Training data missing values: 0
Test data missing values: 0


## Remove Non-informative Columns

In [148]:
titanic_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

The columns I will drop are the PassengerId and the Ticket. Although, I've realised to submit solutions to kaggle I will need the PassengerId column for the test set.

In [149]:
test_ids = titanic_test['PassengerId']

In [150]:
titanic_train.drop(['PassengerId', 'Ticket'], axis=1, inplace=True)
titanic_test.drop(['PassengerId', 'Ticket'], axis=1, inplace=True)

Our data sets have now been cleaned and are ready for feature engineering.

In [156]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       891 non-null object
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [155]:
PROCESSED_PATH = Path('../data/processed')
test_ids.to_csv(PROCESSED_PATH/'test_ids.csv', index=False, header='PassengerId')
titanic_train.to_csv(PROCESSED_PATH/'cleaned_training.csv', index=False)
titanic_test.to_csv(PROCESSED_PATH/'cleaned_test.csv', index=False)